In [1]:
from flash_attn.replace import replace_attn_with_flash_attn
print(replace_attn_with_flash_attn)

ModuleNotFoundError: No module named 'flash_attn.replace'

In [ ]:


import json
import time
import numpy as np
from pathlib import Path
from tqdm import tqdm
from datasets import load_dataset  # ensure load_dataset is defined
from transformers import (
    AutoModelForCausalLM, AutoModelForSequenceClassification,
    AutoTokenizer, BitsAndBytesConfig
)
import torch
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"  # GPU
)
model.eval()

# prompt
prompt = "What is computer?"
formatted_prompt = f"""
<｜begin▁of▁sentence｜><｜User｜>{prompt}<｜Assistant｜><think>
"""

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=0.5,
        top_p= 0.9,
    )

generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



<｜User｜>What is computer?<｜Assistant｜><think>
Okay, so I need to figure out what a computer is. I've heard the term before, but I'm not exactly sure what it means. Let me start by breaking it down. I know computers are used for a lot of things, like running software, handling data, and maybe even playing games. But what


In [5]:

import contextlib
import io

ans = 'def remove_Occ(s, ch):\n    s = s.lstrip(ch)\n    s = s.rstrip(ch)\n    return s'
test = ['assert remove_Occ("hello","l") == "heo"',
 'assert remove_Occ("abcda","a") == "bcd"',
 'assert remove_Occ("PHP","P") == "H"']

def evaluate_generated_code(generated_code, test_cases):
    """
    Evaluate generated code by executing it and running provided test cases.

    Args:
        generated_code (str): Generated Python function code
        test_cases (list of str): List of test expressions to evaluate

    Returns:
        bool: True if all test cases pass, False otherwise
    """
    try:
        # Prepare an isolated execution environment
        exec_globals = {}
        exec_locals = {}

        # Execute the generated code
        with contextlib.redirect_stdout(io.StringIO()):
            exec(generated_code, exec_globals, exec_locals)

        # Now run all test cases
        for test in test_cases:
            # Each test is a string like "assert my_func(5) == 120"
            with contextlib.redirect_stdout(io.StringIO()):
                exec(test, exec_globals, exec_locals)

        return True  # All tests passed

    except Exception as e:
        # If any error happens (syntax error, wrong output, etc.), the function is incorrect
        return False

evaluate_generated_code(ans,test)

False